# Decision Trees and Random Forests

Shaurya Jauhari (Email: shauryajauhari@gzhmu.edu.cn)

Welcome to the second workshop in the series of **Machine Learning Fundamentals**. In this session, we shall explore the theory of **Decision Trees** and scale it to the broader concept of **Random Forests**. Decision trees epitomize the divide and conquer strategy to accomplish classification tasks (although they could also be implemented for regression chores as well)(Quinlan 1986, Rokach 2005). From the previous session on **Logistic Regression**, you may recall that if the response variable is catergorical in nature (factors in R), we aim classification, or else, if the response variable is continuous, it denotes regression. Classification and regression methodologies are both categorized under supervised machine learning. The theme of machine learning was realized in 1950s and since then, coupled with the data deluge and upheaval in computational prowess, has exhibited stronghold in data analysis domain (See figure below).

![Machine Learning Genesis](./ML_Genesis.png)

In order to appreciate the concept of random forests, it is incumbent to learn about decision trees. A decision tree is the unit of random forest. 

There is also an added understanding about dimensionality reduction in decision trees (and random forests as well). If you may recall the same concept in logistic regression, the idea was to penalize "non-performing" features by reducing their coefficients close to(alpha=0:Ridge) or equal to zero(alpha=1:Lasso).(P.S. There is also a provision for replicating the objective of train-test data partitioning, i.e. cross-validation). In decision trees, entropy and information gain parameters are calculated to ascertain the best attribute to split/ partition the tree. It is crucial to engender the tree a definitive structure, else the curse of biasness in decision trees for continuous and nominal data comes to play.   

Random forests enlighten on the dogma of democracy, i.e. *majority wins*. Decision Trees are rudimentary classification algorithms that, at a low-level, are synonymous to *if-then* conditional statements in programming languages. They follow the strategy of iterative recursion, and intuitively the leaf nodes hold the final verdict. The highest aggregate from all leaf nodes (terminals) is graded as the output of that decision tree.  

In this module, we shall delve into creation of basic decision trees to have an understanding of it. For the purpose, we shall load the package **party** and make use of the function **ctree()** to calculate and analyze decision trees. Furthermore, we shall require the packages **rpart**, **party**, **randomForest** as well. Let's load them up.

## Package Installation and Loading

In [1]:
install.packages("party", dependencies = TRUE, repos = "https://mirrors.tuna.tsinghua.edu.cn/CRAN/")
install.packages("rpart", dependencies = TRUE, repos = "https://mirrors.tuna.tsinghua.edu.cn/CRAN/")
install.packages("randomForest", dependencies = TRUE, repos = "https://mirrors.tuna.tsinghua.edu.cn/CRAN/")
library(party)
library(rpart)
library(randomForest)


The downloaded binary packages are in
	/var/folders/hm/c3_fjypn62v5xh5b5ygv267m0000gn/T//RtmpTrphVb/downloaded_packages


Loading required package: grid
Loading required package: mvtnorm
Loading required package: modeltools
Loading required package: stats4
Loading required package: strucchange
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Loading required package: sandwich


Let us now, pick up a dataset. The dataset pertains to the soft computation of the **enhancer prediction** module in bioinformatics. We certainly take into cognizance the biological implications about enhancer regions (See figure below).   
Certain known classes of proteins/ cis-regulatory elements called Transcription Factors (TFs) and Transcription Co-Activators (TCoAs) are programmed to bind to regions in the genome called *Enhancers*, that remotely orchestrate the phenomena of gene regulation. They are at a distal location to the *Promoters*, regions associated with genes and respective Transcription Start Sites (TSS). On stimulus from TFs, the enhancer and promoter sequences reciprocate and actuate the transcription machinery.

![Enhancer Prediction Basis](./WorkFlow_Illustration.jpg)

## Dataset handling

For use in this session, we have chosen a compound dataset from the studies as follows; for **non-enhancer** data, we referenced "Chromatin accessibility assay of IMR90 Cell Line; DS11759", sample dataset deposited to GEO labeled GSM468792. (P.S. Although these are not, but if a  dataset is originally available as a WIG file, firstly, it is converted to BED and is then subsequently cleaned for  use(See Notes: 3).
To add to the positive class data, we consider regions that are **enhancers**. Under this category, we include regions from ChIP-Seq study from GSM929090, available as BED file. These are p300 binding sites from the IMR90 cell lines. Also, in interest of this exercise, we shall consider equal proportions of enhancer and nonenhancer regions; although, enhancer sites are lower in number in comparison to the promoter sites (Rajagopal 2013).

![Dataset Profile](./Dataset_Profile.jpg)

In [4]:
mydata_enhancers <- read.csv("./GSM2445787.bed", sep = '\t', header = FALSE)
mydata_nonenhancers <- read.csv("./GSM817160.bed", sep = '\t', header = FALSE)

We choose to incorporate only those peaks with occurence of 1 and above. Additionally, a column for "Class" has been added that uniformly holds the value "Enhancer" as all the peaks correspond to p300 bindings in the genome. Finally, we have also pruned the dataset for columns on peak ids and frequencies as they are beyond the scope of our analysis.

In [5]:
mydata_enhancers <- mydata_enhancers[mydata_enhancers$V5 > 0,]
mydata_enhancers <- mydata_enhancers[,c(1,2,3)]
mydata_enhancers$Class <- "Enhancer"
colnames(mydata_enhancers) <- c("Chrom", "Start", "End", "Class")

In [6]:
enhancer_indexes <- sample(1:nrow(mydata_enhancers),nrow(mydata_nonenhancers)) 
mydata_enhancers<- mydata_enhancers[enhancer_indexes,]

Again for the nonenhancer data, we shall be following the same protocol.

In [7]:
mydata_nonenhancers <- mydata_nonenhancers[mydata_nonenhancers$V5 > 0,]
mydata_nonenhancers <- mydata_nonenhancers[,c(1,2,3)]
mydata_nonenhancers$Class <- "Non-Enhancer"
colnames(mydata_nonenhancers) <- c("Chrom", "Start", "End", "Class")

We have hit a roadbloack here! In order to execute for decision trees, given the dataset, we shall always have a node
that is having two attributes minimum. Since fields "Chrom", "Start", and "End" are intimately related, we will need 
"Chrom" as default attribute, alongwith "Start" or "End" or both. There are two ways to solve this problem. First is 
to have a tree with two attributes at each node. Second way could help by, (i) having the file in sorted order by 
chromosome names, (ii) and then setting up a new column indicating cumulative "Start" and "End" extremities. 

Let us try the second way out.
It is better to confirm if we have the data in sorted order by chromosomes.

In [8]:
mydata_enhancers_sorted_chrom_names <- mydata_enhancers[with(mydata_enhancers, order(Chrom,Start)), ]
mydata_nonenhancers_sorted_chrom_names <- mydata_nonenhancers[with(mydata_nonenhancers, order(Chrom,Start)), ]

Cool! Eventually, we converge these dataframes into one and *voila* you have a base version of the dataset to work upon. Let's say we name it *my_consolidated_dataset*. 

In [12]:
my_consolidated_dataset <- rbind(mydata_enhancers_sorted_chrom_names, mydata_nonenhancers_sorted_chrom_names)
my_consolidated_dataset <- my_consolidated_dataset[with(my_consolidated_dataset, order(Chrom, Start)), ]

In addition we're also not willing to process random chromosomal entries that might've crept in.

In [14]:
my_consolidated_dataset<- as.data.frame(my_consolidated_dataset[my_consolidated_dataset$Chrom %in% c("chr1","chr2","chr3","chr4","chr5","chr6","chr7",
                                                                       "chr8","chr9","chr10","chr11","chr12","chr13","chr14",
                                                                       "chr15", "chr16", "chr17", "chr18", "chr19", "chr20",
                                                                       "chr21", "chr22", "chrX", "chrY"), ])
my_consolidated_dataset

,Chrom,Start,End,Class
,<fct>,<int>,<int>,<chr>
1,chr1,10153,10350,Non-Enhancer
149,chr1,14800,14900,Enhancer
508,chr1,50700,50800,Enhancer
577,chr1,57600,57700,Enhancer
638,chr1,63700,63800,Enhancer
827,chr1,82600,82700,Enhancer
1018,chr1,101700,101800,Enhancer
1234,chr1,123300,123400,Enhancer
2,chr1,180794,180924,Non-Enhancer


You would've noticed one thing. The class variable is a "character". We would like it to be factor variable. So, let's change it. Note that this manipulation can also be carried out at the time of defining the *Class* variable ,but whatever!

In [ ]:
my_consolidated_dataset$Class <- as.factor(my_consolidated_dataset$Class)

So, the idea here is that we shall have regions in chromosome 1 indexed as is, and then we shall engage the end index of chromosome 1 as the starting index of chromosome 2; same for chromosome 3 of chromosome 2 ans so forth. In order to accomplish that, let us break down the given dataset for each chromosome.

In [15]:
chr1_data<- my_consolidated_dataset[my_consolidated_dataset$Chrom == "chr1", ]
chr2_data<- my_consolidated_dataset[my_consolidated_dataset$Chrom == "chr2", ]
chr3_data<- my_consolidated_dataset[my_consolidated_dataset$Chrom == "chr3", ]
chr4_data<- my_consolidated_dataset[my_consolidated_dataset$Chrom == "chr4", ]
chr5_data<- my_consolidated_dataset[my_consolidated_dataset$Chrom == "chr5", ]
chr6_data<- my_consolidated_dataset[my_consolidated_dataset$Chrom == "chr6", ]
chr7_data<- my_consolidated_dataset[my_consolidated_dataset$Chrom == "chr7", ]
chr8_data<- my_consolidated_dataset[my_consolidated_dataset$Chrom == "chr8", ]
chr9_data<- my_consolidated_dataset[my_consolidated_dataset$Chrom == "chr9", ]
chr10_data<- my_consolidated_dataset[my_consolidated_dataset$Chrom == "chr10", ]
chr11_data<- my_consolidated_dataset[my_consolidated_dataset$Chrom == "chr11", ]
chr12_data<- my_consolidated_dataset[my_consolidated_dataset$Chrom == "chr12", ]
chr13_data<- my_consolidated_dataset[my_consolidated_dataset$Chrom == "chr13", ]
chr14_data<- my_consolidated_dataset[my_consolidated_dataset$Chrom == "chr14", ]
chr15_data<- my_consolidated_dataset[my_consolidated_dataset$Chrom == "chr15", ]
chr16_data<- my_consolidated_dataset[my_consolidated_dataset$Chrom == "chr16", ]
chr17_data<- my_consolidated_dataset[my_consolidated_dataset$Chrom == "chr17", ]
chr18_data<- my_consolidated_dataset[my_consolidated_dataset$Chrom == "chr18", ]
chr19_data<- my_consolidated_dataset[my_consolidated_dataset$Chrom == "chr19", ]
chr20_data<- my_consolidated_dataset[my_consolidated_dataset$Chrom == "chr20", ]
chr21_data<- my_consolidated_dataset[my_consolidated_dataset$Chrom == "chr21", ]
chr22_data<- my_consolidated_dataset[my_consolidated_dataset$Chrom == "chr22", ]
chrX_data<- my_consolidated_dataset[my_consolidated_dataset$Chrom == "chrX", ]
chrY_data<- my_consolidated_dataset[my_consolidated_dataset$Chrom == "chrY", ]

Now that we have clustered all the chromosomes individually, we can proceed towards 

### Notes

  1.  The authors [@Rajagopal2013] construed p300 (a transcription co-activator) binding sites overlapping DNase-I hypersentitive sites and distal to annotated transcription start sites (TSS) as active p300 binding sites representative of enhancers.
  2. Clusters with presence or absence of H3K36me3 were hypothesized to represent genic and inter-genic enhancers respectively.
  3. While pre-processing the genomic ranges data if you need to convert WIG file to BED file, you may want to refer to BEDOPS -> wig2bed() function. This is available as a command line utility.

P.S. During intermediary partitioning, if the node has the lowest Gini Index, it becomes leaf node. That will most likely be the case when all remaining non-root nodes have been exhausted, checking for impurity score.

All workshop study material is available at my github page (https://github.com/shauryajauhari).
